In [14]:
from flask import Flask, request, jsonify, send_file
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import os

app = Flask(__name__)

# Load trained response analysis model (Train it if not done)
response_analyzer = joblib.load("C:/Users/Priti V/collegeproject/ML/models/student_response_analyzer.pkl")

# Ensure directory exists for reports
os.makedirs("static/reports", exist_ok=True)

@app.route("/analyze_response", methods=["POST"])
def analyze_response():
    data = request.json  # Expecting a JSON payload with 'student_id' and 'responses'
    student_id = data.get("student_id")
    responses = pd.DataFrame(data.get("responses"))
    
    # Check if correct answers
    responses['Correct'] = responses['Selected Answer'] == responses['Correct Answer']
    
    # Calculate skill performance
    skill_performance = responses.groupby('Skill Type')['Correct'].mean() * 100
    
    # Generate bar chart
    img_path = f"static/reports/student_{student_id}.png"
    skill_performance.plot(kind='bar', color=['blue', 'green', 'red'])
    plt.xlabel("Skills")
    plt.ylabel("Performance (%)")
    plt.title(f"Student {student_id} Performance Report")
    plt.ylim(0, 100)
    plt.savefig(img_path)  # Save image
    plt.close()
    
    return jsonify({"message": "Report generated!", "image_url": f"http://127.0.0.1:5002/{img_path}"})

# Serve static images
@app.route("/static/reports/<filename>")
def get_report_image(filename):
    return send_file(f"static/reports/{filename}", mimetype="image/png")

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5002, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://192.168.29.104:5002
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\Priti V\collegeproject\ML\venv\Lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

# Load student responses dataset
df = pd.read_csv("C:/Users/Priti V/collegeproject/ML/data/raw/student_responses.csv")

def analyze_student_performance(student_id):
    """
    Analyze student responses and generate a performance report.
    """
    # Filter data for the given student
    student_data = df[df["Student ID"] == student_id]

    if student_data.empty:
        print(f"No data found for Student ID: {student_id}")
        return

    # Check correctness of answers
    student_data["Correct"] = student_data["Selected Answer"] == student_data["Correct Answer"]

    # Calculate performance by skill type
    skill_performance = student_data.groupby("Skill Type")["Correct"].mean() * 100

    # Plot bar chart for visualization
    plt.figure(figsize=(8, 5))
    sns.barplot(x=skill_performance.index, y=skill_performance.values, palette="viridis")
    plt.xlabel("Skill Type")
    plt.ylabel("Accuracy (%)")
    plt.title(f"Student Performance Report (ID: {student_id})")
    plt.ylim(0, 100)
    plt.xticks(rotation=45)

    # Save the generated report
    os.makedirs("static/reports", exist_ok=True)
    report_path = f"static/reports/student_{student_id}_report.png"
    plt.savefig(report_path)
    plt.close()

    print(f"Report generated and saved at: {report_path}")
    return report_path

# Example usage:
# analyze_student_performance(student_id=101)
